In [45]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import math
import random


def SoftMax(values):
    answers = [0]*len(values)
    denom = 0
    for i in range(len(values)):
        denom += math.exp(values[i])
    for i in range(len(values)):
        answers[i] = math.exp(values[i])/denom
    return answers

def Sigmoid(values): #This function does sigmoid on all the values, not just one at a time
    answers = [0]*len(values)
    for i in range(len(values)):
        answers[i] = 1/(1+math.exp(-values[i]))
    return answers



def OneHot(label):
    ans = [0]*10
    ans[label] = 1
    return ans


class neuralnet:

    def __init__(self, numinputs, numoutputs, errorfunc="MSE"):

        self.numinputs = numinputs
        self.numoutputs = numoutputs
        self.errorfunc = errorfunc
        
        self.biases = np.random.default_rng().random(self.numoutputs)
        self.weights = np.random.default_rng().random((self.numoutputs, self.numinputs))

        if self.errorfunc == "MSE":
            self.biases /= 1000
            self.weights /= 1000


    def evaluate(self, inputs):
            try:
                output = self.weights @ inputs + self.biases
                return output
            except:
                print("Dimension mismatch!")

    def computeerror(self, outputs, trueoutputs):
        if self.errorfunc == "MSE":

            sum = 0
            for i in range(len(outputs)):
                sum += (outputs[i] - trueoutputs[i])**2
            sum = sum * (1/(2*self.numoutputs))
            return sum 

        if self.errorfunc == "CrossEntropy":
            sum = 0
            for i in range(0, len(outputs)):
                sum += -trueoutputs[i]*math.log2(outputs[i])
            return sum


    def computegradient(self, inputs, labels):

        wparts = np.zeros((self.numoutputs, self.numinputs))
        bparts = np.zeros(self.numoutputs)


        ys = self.weights @ inputs + self.biases # We need this regardless of the error function
        M3 = np.zeros((self.numoutputs, self.numoutputs, self.numinputs)) # Ditto
        for i in range(self.numoutputs):
            for j in range(self.numinputs):
                M3[i,i,j] = inputs[j]

        if self.errorfunc == "MSE":
            
            # You need to put stuff here

            zs = Sigmoid(ys)

            M2 = np.zeros((self.numoutputs, self.numoutputs))
            for i in range(self.numoutputs):
                for j in range(self.numoutputs):
                    if (i == j):
                        M2[i,j] = math.exp(-ys[1])/(1+math.exp(-ys[1]))**2

            M1 = np.zeros((self.numoutputs))
            for i in range(self.numoutputs):
                M1[i] = 1 / self.numoutputs * (zs[i] - labels[i])
        

            wparts = M1 @ (M2 @ M3)
            bparts = M1 @ M2


            return (wparts, bparts)


        if self.errorfunc == "CrossEntropy":

            softdenom = 0

            for i in range(len(ys)):
                softdenom += math.exp(ys[i])

            zs = SoftMax(ys)
            

            M2 = np.zeros((self.numoutputs, self.numoutputs))
            for i in range(self.numoutputs):
                for j in range(self.numoutputs):
                    if (i == j):
                        M2[i,j] = ((softdenom)*math.exp(ys[j]) - math.exp(2*ys[j])) / (softdenom**2)
                    else:
                        M2[i,j] = -(math.exp(ys[i]+ys[j])) / (softdenom ** 2)

            M1 = np.zeros((self.numoutputs))
            for i in range(self.numoutputs):
                M1[i] = -labels[i] / (zs[i] * math.log(2))
        

            wparts = M1 @ (M2 @ M3)
            bparts = M1 @ M2
            
            return (wparts, bparts)

       

        
    
    def updateweights(self, wparts, bparts, eta):

        # This doesn't need to be changed, since all it takes in are
        # the partial derivatives computed elsewhere

        self.weights -= eta*wparts
        self.biases -= eta*bparts



In [22]:
#Instantiate network and set weights/biases
NNAssignment4 = neuralnet(2,2,"CrossEntropy")
NNAssignment4.weights = np.array([[0.1, 0.2], [0.3, 0.4]])
NNAssignment4.biases = np.array([0.7, 0.8])


print(NNAssignment4.weights)
print(NNAssignment4.biases)

NNAssignment4.weights @ np.array([0.5, 0.6]) + NNAssignment4.biases


[[0.1 0.2]
 [0.3 0.4]]
[0.7 0.8]


array([0.87, 1.19])

In [23]:
(wparts, bparts) = NNAssignment4.computegradient([0.5,0.6],[1,0])

print(wparts)
print(bparts)

[[-0.41789411 -0.50147294]
 [ 0.41789411  0.50147294]]
[-0.83578823  0.83578823]


In [46]:
# Assignment 5 testing

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Define two versions of our neural net

MyNeuralNet1 = neuralnet(784,10,"CrossEntropy")
MyNeuralNet2 = neuralnet(784,10,"MSE")


image = (train_images[0].astype(float)/255).flatten()
label = OneHot(train_labels[0])
(wparts, bparts) = MyNeuralNet2.computegradient(image, label)

output = MyNeuralNet2.evaluate(image)

print(Sigmoid(output))

print(np.matrix(wparts).max())



[0.512946920393323, 0.5132761714827284, 0.5142314800036145, 0.5136748669706976, 0.5129043715096221, 0.513605022825544, 0.5127035476962976, 0.5153045697172077, 0.5122467562710654, 0.5133212103467188]
0.012873531653128179


In [62]:
# Run this cell once, to train each network over 10 epochs.  Then run the next cell to compute the errors.
# THEN: Re-run this cell and the next cell to see how the error decreases for each NN.

# Train Neural Net 1, 1000 images, over 10 epochs

for j in range(10):
    for i in range(len(train_images)):
        image = (train_images[i].astype(float)/255).flatten()
        label = OneHot(train_labels[i])
        (wparts, bparts) = MyNeuralNet1.computegradient(image, label)
        MyNeuralNet1.updateweights(wparts, bparts, 0.01)


# Train Neural Net 2, 1000 images, over 10 epochs

for j in range(10):
    for i in range(len(train_images)):
        image = (train_images[i].astype(float)/255).flatten()
        label = OneHot(train_labels[i])
        (wparts, bparts) = MyNeuralNet2.computegradient(image, label)
        MyNeuralNet2.updateweights(wparts, bparts, 1)

In [75]:
cumulativeerror1 = 0
cumulativeerror2 = 0

for i in range(1000):
    test1 = (test_images[i].astype(float)/255).flatten()
    label1 = OneHot(test_labels[i])
    out1 = SoftMax(MyNeuralNet1.evaluate(test1))
    cumulativeerror1 += MyNeuralNet1.computeerror(out1, label1)

for i in range(1000):
    test2 = (test_images[i].astype(float)/255).flatten()
    label2 = OneHot(test_labels[i])
    out2 = Sigmoid(MyNeuralNet2.evaluate(test2))
    cumulativeerror2 += MyNeuralNet2.computeerror(out2, label2)

#print(cumulativeerror1)
#print(cumulativeerror2)


#print(test_labels[0])
test2 = Sigmoid(MyNeuralNet2.evaluate((test_images[0].astype(float)/255).flatten()))
test3 = SoftMax(MyNeuralNet1.evaluate((test_images[0].astype(float)/255).flatten()))
#print(test2)
#print(test3)

MyNeuralNet2.computeerror(test2, OneHot(test_labels[0]))


test2.index(np.matrix(test2).max())


7

In [76]:
# Compute pass rate

passrate1 = 0
passrate2 = 0

for i in range(len(test_images)):

    result1 = SoftMax(MyNeuralNet1.evaluate((test_images[i].astype(float)/255).flatten()))
    result2 = Sigmoid(MyNeuralNet2.evaluate((test_images[i].astype(float)/255).flatten()))
    label = OneHot(test_labels[i])

    if result1.index(np.matrix(result1).max()) == test_labels[i]:
        passrate1 += 1
    if result2.index(np.matrix(result2).max()) == test_labels[i]:
        passrate2 += 1
    

print("Pass Rate 1: ", passrate1/len(test_images))
print("Pass Rate 2: ", passrate2/len(test_images))




Pass Rate 1:  0.8474
Pass Rate 2:  0.61
